# Classification of cats and dogs images by fine tuning the pre trained lightweight Mobilenet CNN model for 2 classes instead of 1000 original classes, by cutting off last 5 layers and adding one last dense layer. Model trained on 1200 images, validated on 400 and accuracy achieved on 400 test images is 0.99.

# Model was trained on Kaggle GPU using 1600 images and 30 epochs. It was immaterial after just 5 epochs due to pre trained model, accuracy was 100%.

In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model, model_from_json, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy, binary_crossentropy
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from tensorflow.keras import regularizers
from tensorflow.keras.metrics import categorical_crossentropy, sparse_categorical_crossentropy, binary_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import imagenet_utils, mobilenet
from tensorflow.keras.preprocessing import image
import itertools
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image

import os

os.listdir('/home/sandeep/Development/Datasets/Nature/less_cats_and_dogs/valid/')

In [ ]:
# Save the filepaths for the datasets
# train_path = '/home/sandeep/Development/Datasets/Nature/less_cats_and_dogs/train/'
# valid_path = '/home/sandeep/Development/Datasets/Nature/less_cats_and_dogs/valid/'
# test_path = '/home/sandeep/Development/Datasets/Nature/less_cats_and_dogs/test/'

In [2]:
# these paths were used on kaggle after uploading the dataset
train_path = '../input/2000-cats-dogs/more_cats_dogs/train'
valid_path = '../input/2000-cats-dogs/more_cats_dogs/valid'
test_path = '../input/2000-cats-dogs/more_cats_dogs/test'

In [11]:
%config Completer.use_jedi = False

In [3]:
# Load the data generators, also preprocess the inputs
train_batches = ImageDataGenerator(preprocessing_function=mobilenet.preprocess_input
                                  ).flow_from_directory(train_path, target_size=(224,224), 
                                classes=['cats', 'dogs'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=mobilenet.preprocess_input
                                  ).flow_from_directory(valid_path, target_size=(224,224), 
                                classes=['cats', 'dogs'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=mobilenet.preprocess_input
                                 ).flow_from_directory(test_path, target_size=(224,224), 
                                classes=['cats', 'dogs'], batch_size=10, shuffle=False)

In [4]:
# Load the mobilenet model
model = mobilenet.MobileNet()
model.summary()

In [5]:
# Save the original mobilenet model for downloading and saving on disk later
model.save('mobilenet_original.h5')

In [6]:
# Grab the output of the sixth last layer of the mobilenet original model, 
# and add a last dense layer with 2 classes. This will cut off the last 5 layers of 
# the original model
x = model.layers[-6].output
predictions = Dense(2, activation='softmax')(x)
model2 = Model(inputs=model.input, outputs=predictions)

In [7]:
model2.summary()

In [8]:
# Now we will only train the last 5 layers of the model and freeze all other layer parameters
for layer in model2.layers[:-5]:
    layer.trainable = False
model2.trainable_variables

## So the new model is tuned and ready to be trained on cats and dogs

In [9]:
# Compile the new model
model2.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', 
               metrics=['accuracy'])

In [10]:
# Train the new model
model2.fit(train_batches, steps_per_epoch=120, validation_data=valid_batches, 
                    validation_steps=40, epochs=30, verbose=2)

In [12]:
# Save the new tuned and trained model
model2.save('mobilenet_last5_2000catsdogs.h5')

In [13]:
# Get the labels of the test dataset
test_labels = test_batches.classes
test_labels

In [14]:
# Get the classes of the test dataset
test_batches.class_indices

In [15]:
preds = model2.predict(test_batches, steps=40, verbose=1)

In [18]:
ConfusionMatrixDisplay.from_predictions(test_labels, preds.argmax(axis=1), labels=[0, 1], 
                                        display_labels=['Cat', 'Dog'])

In [20]:
accuracy_score(test_labels, preds.argmax(axis=1))

# The model has performed rather miraculously with precision 199/202 for Cat and recall 199/200 for Cat, with accuracy 0.99.